In [1]:
# DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# XGB
import xgboost as xgb

# cross_val_score
from sklearn.model_selection import cross_val_score

# Import pandas
import pandas as pd


from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

from pyspark.sql.functions import desc,asc, monotonically_increasing_id, split, size, length

# Plot AUC *(Self Defined Package)
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
%run /Users/p0z00cx/jupter/product_weather_prediction/lib/production_utility.ipynb

In [3]:
%run /Users/p0z00cx/jupter/product_weather_prediction/lib/roc_auc.ipynb

In [6]:
test_data_set = spark.read.csv('/Users/p0z00cx/hactron_product/test_data_set_final.csv', header="true", inferSchema="true")
display(test_data_set.toPandas())

,item_desc,dc_name,prcp_aver,tavg,result,dept_desc_new,state_new
0,BLACKBERRY 12OZ,"PAULS VALLEY, OK GDC",0.0,70.000000,0,BLACKBERRIES,California
1,BLACKBERRY 12OZ,"CLARKSVILLE, AR GDC",0.0,70.000000,1,BLACKBERRIES,California
2,STRAWBERRY 2#,"AUBURN, IN GDC",0.0,70.000000,1,STRAWBERRIES,Michoacan
3,STRAWBERRY 2#,"COLTON, CA FRESH",0.0,70.000000,1,STRAWBERRIES,California
4,APPLE FUJI 3# 8,"BEDFORD, PA GDC",0.0,156.473684,1,APPLES,Pennsylvania
...,...,...,...,...,...,...,...
26165,AVO HASS 72 MX LGS,"BEDFORD, PA GDC",0.0,70.000000,1,AVOCADOS,Michoacan
26166,STRAWBERRY 1# SB,"NEW ALBANY, MS GDC",0.0,70.000000,1,STRAWBERRIES,None
26167,STRAWBERRY 2# SB,"MANKATO, MN GDC",0.0,70.000000,1,STRAWBERRIES,None
26168,PMPKN BIN 40 IHP,"BRUNDIDGE, AL GDC",0.0,70.000000,1,None,None


In [7]:
df = test_data_set.toPandas()
df.isna().sum()

item_desc           0
dc_name             0
prcp_aver           0
tavg                0
result              0
dept_desc_new     908
state_new        3080
dtype: int64

In [8]:
X= df.loc[:,['dept_desc_new','state_new','dc_name','prcp_aver','tavg']]
X = pd.get_dummies(X, prefix_sep='_', drop_first=True)
X

,prcp_aver,tavg,dept_desc_new_AVOCADOS,dept_desc_new_BLACKBERRIES,dept_desc_new_BLUEBERRIES,dept_desc_new_LIMES,dept_desc_new_ORG MIXED BERRIES,dept_desc_new_PEARS,dept_desc_new_RASPBERRIES,dept_desc_new_STRAWBERRIES,...,"dc_name_ROBERT, LA GDC","dc_name_SHELBYVILLE, TN GDC","dc_name_STERLING, IL GDC","dc_name_TEMPLE, TX GDC","dc_name_TERRELL, TX GDC","dc_name_TOMAH, WI GDC","dc_name_VINELAND, NJ FRESH","dc_name_WASHINGTON CRT HO, OH GDC","dc_name_WINTER HAVEN, FL GDC","dc_name_WINTERSVILLE, OH GDC"
0,0.0,70.000000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,70.000000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,70.000000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.0,70.000000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.0,156.473684,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26165,0.0,70.000000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26166,0.0,70.000000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
26167,0.0,70.000000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
26168,0.0,70.000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
Y = df.result  
Y

0        0
1        1
2        1
3        1
4        1
        ..
26165    1
26166    1
26167    1
26168    1
26169    1
Name: result, Length: 26170, dtype: int32

In [12]:
# Instantiate XGBClassifier
from sklearn.metrics import classification_report
xgb_cl = xgb.XGBClassifier()
clf_1 = xgb_cl.fit(X,Y)
pred_y_1 = clf_1.predict(X)
target_names = ['result 0', 'result 1']
print (classification_report( Y,pred_y_1, target_names=target_names))

              precision    recall  f1-score   support

    result 0       0.73      0.34      0.47      4702
    result 1       0.87      0.97      0.92     21468

    accuracy                           0.86     26170
   macro avg       0.80      0.66      0.69     26170
weighted avg       0.85      0.86      0.84     26170

